## Transfer Learning

### import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.backend import clear_session

### VGG16 model

In [2]:
IMAGE_SIZE = [224,224]
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [3]:
#dont train existing weight
for layer in vgg.layers:
    layer.trainable = False

In [4]:
folder = glob('pneumonia_data/train/*')
#we need to add more data
x = Flatten()(vgg.output)
prediction = Dense(len(folder), activation = tf.nn.softmax)(x)
model = Model(inputs = vgg.input, outputs = prediction)

In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [6]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

In [8]:
#import image from datasets using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  featurewise_center = False, #set input mean 0 over the data
                                  samplewise_center = False, #set each sample mean 0
                                  featurewise_std_normalization=False, #divide inputs by std of the datasets
                                  zca_whitening = False,
                                  rotation_range=10, #randomly rotate images in the range 0-100
                                  width_shift_range = 0.2, #randomly shift image vertically. fraction of total height.
                                  vertical_flip = False
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('pneumonia_data/train',
                                                target_size=(128,128),
                                                batch_size=32,
                                                class_mode='categorical')

test_set = test_datagen.flow_from_directory('pneumonia_data/test',
                                            target_size=(128,128),
                                            batch_size=32,
                                            class_mode='categorical')

Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [21]:
#train model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=5232//32,
  validation_steps=len(test_set)
)

Epoch 1/5
32/32 [==============================] - 624s 19s/step - loss: 0.3368 - accuracy: 0.8685 - val_loss: 0.1650 - val_accuracy: 0.9394
Epoch 2/5
32/32 [==============================] - 572s 18s/step - loss: 0.2383 - accuracy: 0.9243 - val_loss: 0.1555 - val_accuracy: 0.9091
Epoch 3/5
32/32 [==============================] - 512s 16s/step - loss: 0.1955 - accuracy: 0.9363 - val_loss: 0.1442 - val_accuracy: 0.9242
Epoch 4/5
32/32 [==============================] - 580s 18s/step - loss: 0.1577 - accuracy: 0.9562 - val_loss: 0.1352 - val_accuracy: 0.9242
Epoch 5/5
32/32 [==============================] - 620s 19s/step - loss: 0.0623 - accuracy: 0.9841 - val_loss: 0.1126 - val_accuracy: 0.9545


In [22]:
model.save('model.h5')

#### test our model

In [19]:
def predict(image_path):
    model = load_model('model.h5')
    img = image.load_img(image_path, target_size=(224,224))
    #processing the image
    x = image.img_to_array(img)
    x = x/255  #scaling
    x = np.expand_dims(x, axis=0)
    
    x = preprocess_input(x)
    prediction = model.predict(x)
    clear_session()
    prediction = np.argmax(prediction, axis=1)
    return prediction

In [26]:
#image_path ='covid.png'
#image_path = 'Normal.jpeg'
image_path = '0113.jpeg'
predict(image_path)[0]

0